# Figure 6 Analysis (Section 11.5)

This notebook reproduces the analysis of Figure 6 from Chapter 11 (Model Risk).  We compute default probabilities for:
- CDO tranche‐2 (one level of CDO)  
- CDO² tranche‐2 (two levels: CDO of CDOs)  
under three scenarios:
1. Statistically independent mortgage defaults
2. Perfectly correlated defaults
3. A mixture model combining the two regimes with specified probabilities

We then compute the resulting probabilities and relevant ratios/log‐differences as in the spreadsheet.

## 1️⃣ Import libraries

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import binom

## 2️⃣ Define parameters and compute probabilities under each scenario

We follow the setup from the Excel spreadsheet:
1. Individual mortgage default probability: $p = 0.05$
2. Pool size (number of mortgages in each CDO): $N = 2$
3. Mixture weights:
   - Independent regime: $w_{\mathrm{ind}} = 0.99$
   - Perfectly correlated regime: $w_{\mathrm{pc}} = 0.01$

*Under independent defaults*, each mortgage fails independently with probability $p$.
- CDO tranche‐2 default occurs if both mortgages in that CDO default:  $$P_{\mathrm{CDO},\mathrm{ind}} = 1 - \text{BinomialCDF}(k=1; N=2, p).$$- CDO² tranche‐2 default occurs if both CDOs default (treating each CDO’s failure probability as independent):  $$P_{\mathrm{CDO}^2,\mathrm{ind}} = 1 - \text{BinomialCDF}(k=1; N=2, p_{\mathrm{CDO,ind}}).$$
*Under perfect correlation*, all mortgages fail together with probability $p$.
- Hence $P_{\mathrm{CDO},\mathrm{pc}} = p$ and $P_{\mathrm{CDO}^2,\mathrm{pc}} = p$.

*Under the mixture model*:
- We randomly select one of the two regimes: independent (with weight $0.99$) or perfectly correlated (with weight $0.01$).
- Then $P_{\mathrm{CDO,mix}} = w_{\mathrm{ind}}\,P_{\mathrm{CDO,ind}} + w_{\mathrm{pc}}\,P_{\mathrm{CDO,pc}}$,  
  and similarly for $P_{\mathrm{CDO}^2,\mathrm{mix}}$.

In [2]:
# Parameters
p = 0.05         # individual mortgage default probability
N = 2            # number of mortgages per CDO
w_ind = 0.99     # mixture weight for independent regime
w_pc = 0.01      # mixture weight for perfectly correlated regime

# --- Independent regime ---
# Probability that a CDO tranche-2 defaults = P(>=2 failures out of N)
cdo1_ind = 1 - binom.cdf(1, N, p)
# For CDO^2: treat each CDO as an "aggregate loan" with failure probability cdo1_ind, and again N=2
cdo2_ind = 1 - binom.cdf(1, 2, cdo1_ind)

# --- Perfectly correlated regime ---
# All mortgages default together with probability p
cdo1_pc = p
cdo2_pc = p

# --- Mixture model ---
cdo1_mix = w_ind * cdo1_ind + w_pc * cdo1_pc
cdo2_mix = w_ind * cdo2_ind + w_pc * cdo2_pc

## 3️⃣ Assemble results into a table

In [4]:
# Create a summary DataFrame matching the spreadsheet layout
df_fig6_results = pd.DataFrame({
    'Correlation Structure': ['Independent', 'Perfectly Correlated', 'Mixture'],
    'P(Structure)': [w_ind, w_pc, np.nan],
    'CDO Tranche-2': [cdo1_ind, cdo1_pc, cdo1_mix],
    'CDO^2 Tranche-2': [cdo2_ind, cdo2_pc, cdo2_mix]
})

df_fig6_results

,Correlation Structure,P(Structure),CDO Tranche-2,CDO^2 Tranche-2
0,Independent,0.99,0.002500,0.000006
1,Perfectly Correlated,0.01,0.050000,0.050000
2,Mixture,NaN,0.002975,0.000506


## 4️⃣ Compute ratios and log‐differences

Finally, we compute:
1. The ratio $\displaystyle R_1 = \frac{P_{\mathrm{CDO}^2,\mathrm{mix}}}{P_{\mathrm{CDO,ind}}}$, showing how the mixed CDO² default probability compares to the purely independent CDO default probability.
2. The ratio $\displaystyle R_2 = \frac{P_{\mathrm{CDO}^2,\mathrm{mix}}}{P_{\mathrm{CDO,mix}}}$, as in the spreadsheet.
3. The log‐difference $\ln\bigl(P_{\mathrm{CDO}^2,\mathrm{mix}}\bigr) - \ln\bigl(P_{\mathrm{CDO,mix}}\bigr)$.


In [7]:
# Ratio R1 = cdo2_mix / cdo2_ind
R1 = cdo2_mix / cdo2_ind
# Ratio R2 = cdo2_mix / cdo1_mix
R2 = cdo2_mix / cdo1_mix
# Log-difference
log_diff = np.log(cdo2_mix) - np.log(cdo1_mix)

print(f"R1 = cdo2_mix / cdo1_ind = {R1:.6f}")
print(f"R2 = cdo2_mix / cdo1_mix = {R2:.6f}")
print(f"ln(cdo2_mix) - ln(cdo1_mix) = {log_diff:.6f}")

R1 = cdo2_mix / cdo1_ind = 80.990000
R2 = cdo2_mix / cdo1_mix = 0.170147
ln(cdo2_mix) - ln(cdo1_mix) = -1.771092
